In [ ]:
import requests
import configparser
from datetime import datetime

In [ ]:
## Util functions
def parse_config(config_file):
    config = configparser.ConfigParser()
    config.read(config_file)
    return config

In [ ]:
config = parse_config('config.cfg')
weather_api_key = config['DEFAULT']['open_weather_key']

In [ ]:
## Open Weather functions
## TODO: Find rain probability

date = '11 Oct'
    
def time_slot_datapoint(openweather_response, starting_timings):
    for datapoint in openweather_response['list']:
        if datetime.fromtimestamp(datapoint['dt']).hour in starting_timings:
            return(datapoint)

def morning_datapoint(openweather_response):
    # 5,6,7 are the hours that will fit a 3-hour window at the morning commute time
    return time_slot_datapoint(openweather_response, [5,6,7])

def afternoon_datapoint(openweather_response):
    # 16,17,18 are the hours that will fit a 3-hour window at the afternoon commute time
    return time_slot_datapoint(openweather_response, [16,17,18])
  
def extract_timeslot_weather_data(datapoint):
    weather = {}
    weather['dt'] = datetime.fromtimestamp(datapoint['dt'])
    weather['min_temperature'] = datapoint['main']['temp_min']
    weather['max_temperature'] = datapoint['main']['temp_max']
    weather['humidity'] = datapoint['main']['humidity']
    weather['wind'] = datapoint['wind']['speed']
    return weather
    
def weather_data():
    ## Sunrise / Sunset data
    base_url = 'http://api.openweathermap.org/data/2.5/weather'
    city = 'London,uk'
    request = base_url+'?q={}&appid={}&units=metric'.format(city, weather_api_key)
    data = requests.get(request).json()
    day_weather = {}
    day_weather['sunrise'] = datetime.fromtimestamp(data['sys']['sunrise'])
    day_weather['sunset'] = datetime.fromtimestamp(data['sys']['sunset'])
    
    ## Forecast 3-hour windows data
    base_url = 'http://api.openweathermap.org/data/2.5/forecast'
    city = 'London,uk'
    request = base_url+'?q={}&appid={}&units=metric'.format(city, weather_api_key)
    data = requests.get(request).json()
    weather = {}
    weather['morning'] = {'sunrise': day_weather['sunrise']}
    weather['morning'].update(extract_timeslot_weather_data(morning_datapoint(data)))
    weather['afternoon'] = {'sunset': day_weather['sunset']}
    weather['afternoon'].update(extract_timeslot_weather_data(afternoon_datapoint(data)))
    
    return weather

In [ ]:
weather_data()

In [ ]:
# print("Weather details for {}".format(date))
# print("{} - {} degrees with {}% of humidity".format(min_temperature, max_temperature, humidity))
# print("{} kph winds".format(wind))
# print("Sunrise at {} and Sunset at {}".format(sunrise.strftime("%H:%M:%S"),
#                                               sunset.strftime("%H:%M:%S")))

In [ ]:
## Twitter API helper functions

def authorise_twitter_api(config):
    auth = OAuthHandler(config['DEFAULT']['consumer_key'], 
                        config['DEFAULT']['consumer_secret'])
    auth.set_access_token(config['DEFAULT']['access_token'], 
                          config['DEFAULT']['access_secret'])
    return auth


In [ ]:
import tweepy
from tweepy import OAuthHandler

config = parse_config('config.cfg')
auth = authorise_twitter_api(config)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
## Tweet
api.update_status("This is my first tweet. Hello World!")